In [1]:
from tensorflow import keras
import tensorflow as tf

import numpy as np
import pandas as pd

import tensorflow_text as text
import tensorflow_hub as hub


In [2]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [3]:
df = pd.read_json("Sarcasm_Headlines_Dataset_v2.json", lines=True)


In [4]:
x=df['headline']
y=df['is_sarcastic']

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)


In [5]:
from sklearn.utils import shuffle
x_train, y_train = shuffle(x_train,y_train, random_state=42)
x_test, y_test = shuffle(x_test,y_test, random_state=42)



In [6]:
 
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string,)
preprocessed_text = bert_preprocess(text_input)
encoded_text = bert_encoder(preprocessed_text)

 
Layer = tf.keras.layers.Dense(8, activation='relu')(encoded_text['pooled_output'])
Layer = tf.keras.layers.Dense(1, activation='sigmoid')(Layer)

 
model = tf.keras.Model(inputs=[text_input], outputs = [Layer])

In [7]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [8]:
model.fit(x_train,y_train,epochs=5)

Epoch 1/5
716/716 [==============================] - 253s 340ms/step - loss: 0.5725 - accuracy: 0.7145
Epoch 2/5
716/716 [==============================] - 247s 344ms/step - loss: 0.4811 - accuracy: 0.7748
Epoch 3/5
716/716 [==============================] - 245s 342ms/step - loss: 0.4530 - accuracy: 0.7902
Epoch 4/5
716/716 [==============================] - 242s 338ms/step - loss: 0.4379 - accuracy: 0.7965
Epoch 5/5
716/716 [==============================] - 242s 338ms/step - loss: 0.4221 - accuracy: 0.8021


In [9]:
model.evaluate(x_test,y_test)

179/179 [==============================] - 62s 339ms/step - loss: 0.4284 - accuracy: 0.8003


[0.4284304976463318, 0.8003144860267639]

In [10]:
df_val=pd.read_json('Sarcasm_Headlines_Dataset.json',lines=True)
x_val=df_val['headline']
y_val=df_val['is_sarcastic']

In [12]:
model.evaluate(x_val,y_val)

835/835 [==============================] - 282s 337ms/step - loss: 0.4060 - accuracy: 0.8156


[0.40601593255996704, 0.8156426548957825]

In [13]:
model.save("Bert_sarcasm.h5")